In [0]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost-1.csv', name='media_prediction_and_its_cost-1.csv', size=16667988, modificationTime=1682550968000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost-2.csv', name='media_prediction_and_its_cost-2.csv', size=16667988, modificationTime=1682551383000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost.csv', name='media_prediction_and_its_cost.csv', size=16667988, modificationTime=1682548452000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost_NEW-1.csv', name='media_prediction_and_its_cost_NEW-1.csv', size=14592433, modificationTime=1682602944000),
 FileInfo(path='dbfs:/FileStore/tables/media_prediction_and_its_cost_NEW.csv', name='media_prediction_and_its_cost_NEW.csv', size=14592433, modificationTime=1682555895000)]

In [0]:
# df_raw = pd.read_csv("/Users/benmeiga/Downloads/media prediction and its cost.csv")

# File location and type
file_location = "/FileStore/tables/media_prediction_and_its_cost-1.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df_raw = df.toPandas()

In [0]:
num_cols, cate_cols, pre_dum_cols = [], [], []
for c in df_raw.columns:
    uni_len = len(set(df_raw[c]))
    dtype = df_raw[c].dtypes

    if dtype == object:
        cate_cols.append(c)
    else: # dtype == float
        if uni_len == 2:
            pre_dum_cols.append(c)
        elif uni_len < 10:
            num_cols.append(c)
            print("Number of unique values: %d \t Datatype: %s \t Column name: %s" %(uni_len, dtype, c))
        else:
            num_cols.append(c)

Number of unique values: 6 	 Datatype: float64 	 Column name: unit_sales(in millions)
Number of unique values: 6 	 Datatype: float64 	 Column name: total_children
Number of unique values: 5 	 Datatype: float64 	 Column name: avg_cars_at home(approx)15
Number of unique values: 6 	 Datatype: float64 	 Column name: num_children_at_home
Number of unique values: 5 	 Datatype: float64 	 Column name: avg_cars_at home(approx)18


In [0]:
df_raw

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,...,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4.0,Bag Stuffers,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3.0,Cash Register Lottery,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",59.86
2,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2.0,High Roller Savings,USA,S,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,"Daily Paper, Radio",84.16
3,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2.0,Cash Register Lottery,USA,M,F,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,In-Store Coupon,95.78
4,Breakfast Foods,Frozen Foods,Food,4.08,1.4280,3.0,Double Down Sale,USA,M,M,...,18670.0,5415.0,3610.0,1.0,1.0,1.0,1.0,1.0,Radio,50.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60423,Specialty,Carousel,Non-Consumable,2.76,1.3248,1.0,You Save Days,USA,M,F,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,In-Store Coupon,95.25
60424,Specialty,Carousel,Non-Consumable,1.60,0.4960,1.0,Price Cutters,USA,S,F,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,Sunday Paper,69.42
60425,Specialty,Carousel,Non-Consumable,5.52,2.5392,2.0,Weekend Markdown,USA,M,M,...,15321.0,4294.0,2863.0,1.0,0.0,0.0,0.0,0.0,"Sunday Paper, Radio, TV",67.51
60426,Specialty,Carousel,Non-Consumable,8.28,2.5668,3.0,Sales Days,Canada,S,M,...,27463.0,4193.0,2795.0,1.0,0.0,0.0,0.0,1.0,Sunday Paper,132.88


In [0]:
!pip install requests
!pip install tabulate
!pip install "colorama!"0.3.8"
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/bin/python -m pip install --upgrade pip' command.
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file
     |████████████████████████████████| 840 kB 30.2 MB/s 
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492036 sha256=afda1222b3fc66df8daacff233f12d003970fed22b18cbd8fc9e4a7f24ad2e78
  Stored in directory: /root/.cache/pip/wheels/bf/5d/6a/2e53874f7ec4e2bede522385439531fafec8fafe005b5c3d1b
Successfully built future
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/bin/python -m pip install --upgrade pip' command.
Looking in links: htt

In [0]:
import h2o
from h2o.automl import H2OAutoML

In [0]:
# initialize the h2o session
h2o.init(ip="localhost", port=54444)

Checking whether there is an H2O instance running at http://localhost:54444..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_345"; OpenJDK Runtime Environment (Zulu 8.64.0.19-CA-linux64) (build 1.8.0_345-b01); OpenJDK 64-Bit Server VM (Zulu 8.64.0.19-CA-linux64) (build 25.345-b01, mixed mode)
  Starting server from /local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpq8m4n9x7
  JVM stdout: /tmp/tmpq8m4n9x7/h2o_root_started_from_python.out
  JVM stderr: /tmp/tmpq8m4n9x7/h2o_root_started_from_python.err
  Server is running at http://127.0.0.1:54444
Connecting to H2O server at http://127.0.0.1:54444 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_root_mrkdwf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.971 Gb
H2O_cluster_total_cores:,3
H2O_cluster_allowed_cores:,3
H2O_cluster_status:,"locked, healthy"


In [0]:
# convert pandas dataframe to h2o frame
hf = h2o.H2OFrame(df_raw)
target = "cost"
x = hf.columns
x.remove(target) # remove the target
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=target,  training_frame=hf)
lb = aml.leaderboard # Leader board
print(lb.head(rows=lb.nrows)) # print leader board
aml.leader.download_mojo(path="path/to/mojo.zip")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                     rmse         mse        mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20230427_140917      0.97787    0.956231   0.190005  0.00795499                  0.956231
XGBoost_2_AutoML_1_20230427_140917                        1.04409    1.09012    0.208261  0.00874006                  1.09012
StackedEnsemble_BestOfFamily_1_AutoML_1_20230427_140917   1.05433    1.11161    0.224369  0.0087193                   1.11161
XGBoost_3_AutoML_1_20230427_140917                        1.05653    1.11625    0.280588  0.0088739                   1.11625
XGBoost_1_AutoML_1_20230427_140917                        1.16888    1.36627    0.27742   0.0102081                   1.36627
GBM_2_AutoML_1_20230427_140917                      

In [0]:
!pip install --upgrade mlflow

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/bin/python -m pip install --upgrade pip' command.


In [0]:
import mlflow.h2o
import h2o
from h2o.automl import H2OAutoML
from h2o.grid.grid_search import H2OGridSearch

# Load data into H2OFrame
hf = h2o.H2OFrame(df_raw)

# Define target and predictor variables
target = "cost"
x = hf.columns
x.remove(target)

# Define hyperparameters to search over
hyper_params = {
    'max_models': [5, 10, 15],
    'max_runtime_secs': [30, 60, 120]
}

# Use GridSearch to search over hyperparameter space
aml = H2OAutoML(max_models=10, seed=1)
grid = H2OGridSearch(aml, hyper_params)
grid.train(x=x, y=target, training_frame=hf)

# Get the best model from the grid search
best_model = grid.get_grid()[0]

# Log the best model as an MLflow model
with mlflow.start_run(experiment_id = experiment_id):
    mlflow.log_param("max_models", best_model.params['max_models'])
    mlflow.log_param("max_runtime_secs", best_model.params['max_runtime_secs'])
    mlflow.log_metric("rmse", best_model.rmse())
    mlflow.h2o.log_model(best_model, "my_model")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-2417040963743886> in <cell line: 22>()
     20 # Use GridSearch to search over hyperparameter space
     21 aml = H2OAutoML(max_models=10, seed=1)
---> 22 grid = H2OGridSearch(aml, hyper_params)
     23 grid.train(x=x, y=target, training_frame=hf)
     24 

/local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/lib/python3.9/site-packages/h2o/utils/metaclass.py in __init__(self, *args, **kwargs)
    332         @wraps(clz.__init__)
    333         def __init__(self, *args, **kwargs):
--> 334             super(new_clz, self).__init__(*args, **kwargs)
    335             self._bci = {name: val.__get__(self, new_clz) if callable(val) else val for name, val in clz._bc._instance_attrs.items()}
    336 

/local_disk0/.ephemeral_nfs/envs/pythonEnv-c4a43444-0770-4344-a691-cf0c4c645aea/lib/python3.9/site-package

In [0]:
# MLflow connection
import os
import numpy as np
import matplotlib.pyplot as plt
import mlflow
from mlflow.models import make_metric
 
# creating the evaluation dataframe
eval_data = df_raw.copy()
eval_data["target"] = df_raw[target]
 
 
def squared_diff_plus_one(eval_df, _builtin_metrics):
    """
    This example custom metric function creates a metric based on the ``prediction`` and
    ``target`` columns in ``eval_df``.
    """
    return np.sum(np.abs(eval_df["prediction"] - eval_df["target"] + 1) ** 2)
 
 
def sum_on_target_divided_by_two(_eval_df, builtin_metrics):
    """
    This example custom metric function creates a metric derived from existing metrics in
    ``builtin_metrics``.
    """
    return builtin_metrics["sum_on_target"] / 2
 
 
def prediction_target_scatter(eval_df, _builtin_metrics, artifacts_dir):
    """
    This example custom artifact generates and saves a scatter plot to ``artifacts_dir`` that
    visualizes the relationship between the predictions and targets for the given model to a
    file as an image artifact.
    """
    plt.clf()
    plt.scatter(eval_df["prediction"], eval_df["target"])
    plt.xlabel("Targets")
    plt.ylabel("Predictions")
    plt.title("Targets vs. Predictions")
    plot_path = os.path.join(artifacts_dir, "example_scatter_plot.png")
    plt.savefig(plot_path)
    plt.close()
    return {"example_scatter_plot_artifact": plot_path}
 
experiment_id = "4410962850347306"
 
with mlflow.start_run(experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(aml, "model")
 
    model_uri = mlflow.get_artifact_uri("model")
    
    result = mlflow.evaluate(
        model=model_uri,
        data=eval_data,
        targets="target",
        model_type="regressor",
        evaluators=["default"],
        custom_metrics=[
            make_metric(
                eval_fn=squared_diff_plus_one,
                greater_is_better=False,
            ),
            make_metric(
                eval_fn=sum_on_target_divided_by_two,
                greater_is_better=True,
            ),
        ],
        custom_artifacts=[prediction_target_scatter],
    )


2023/04/27 14:21:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpbx9d39ob/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.24.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-4410962850347337> in <cell line: 47>()
     50     model_uri = mlflow.get_artifact_uri("model")
     51 
---> 52     result = mlflow.evaluate(
     53         model=model_uri,
     54         data=eval_data,

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/mlflow/models/evaluation/base.py in evaluate(model, data, targets, model_type, dataset_path, feature_names, evaluators, evaluator_config, custom_metrics, custom_artifacts, validation_thresholds, baseline_model, env_manager)
   1284 
   1285     if isinstance(model, str):
-> 1286         model = _load_model_or_server(model, env_manager)
   1287     elif env_manager != _EnvManager.LOCAL:
   1288         raise MlflowException(

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/mlflow/pyfunc/__init__.py in _load_mode